### DSC540 week 5&6 assignment
##### Author: Xin Tang
###### date: 1/28/2024¶

##### Chapter 7 exercise:  Data cleanning and manupulating

In [2]:
#import necessary packages
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import re

In [2]:
# read in the csv file and create dataframe
df=pd.read_excel('candyhierarchy2017.xlsx')
# read first 10 record 
df.head(10)

C:\Users\Daisy\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Internal_ ID,GOING_ OUT,GENDER,AGE,COUNTRY,STATE_PROVINCE_ COUNTY,100_Grand_Bar,Mary_Janes,full-sized-candy-bar,Black_jacks,...,DESPAIR_ OTHER,OTHER_COMMENTS,DRESS,Unnamed: 113,DAY,MEDIA_[Daily_Dish],MEDIA_Science,MEDIA_ ESPN,MEDIA_Yahoo,Coordinates
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"
5,90272852,No,Male,NaN,NaN,NaN,JOY,DESPAIR,JOY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,"(75, 23)"
6,90272853,No,Male,53,usa,Colorado,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,90272854,No,Male,33,canada,ontario,JOY,DESPAIR,JOY,DESPAIR,...,when the little kids get the big chocolate bar...,"first person to give out cans of beer, wins",Blue and black,NaN,Friday,NaN,1.0,NaN,NaN,"(55, 5)"
8,90272858,No,Male,40,Canada,Ontario,JOY,DESPAIR,JOY,MEH,...,NaN,NaN,Blue and black,NaN,Sunday,NaN,1.0,NaN,NaN,"(76, 24)"
9,90272859,No,Female,53,Us,Wa,MEH,DESPAIR,JOY,MEH,...,NaN,NaN,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(70, 28)"


In [3]:
df.shape

(2460, 120)

In [3]:
#There are 2460 rows. for ease of practice, I will only use the first 100 rows.
df2 = df.iloc[:100]

###### Task 1. filter out missing data

In [72]:
# The essentail columns are Q 2 to Q5. so finding missing data for these columns 
print("The column Gender contains NaN - %r " % df2.GENDER.isnull().values.any())
print("The column AGE Address contains NaN - %s " % df2.AGE.isnull().values.any())
print("The column COUNTRY contains NaN - %s " % df2.COUNTRY.isnull().values.any())

The column Gender contains NaN - True 
The column AGE Address contains NaN - True 
The column COUNTRY contains NaN - True 


In [4]:
# remove rows with NaN in these 3 columns
df3 = df2.dropna(subset=['GENDER'])
df4 = df3.dropna(subset=['AGE'])
df5 = df4.dropna(subset=['COUNTRY'])
df6 = df5.dropna(subset=['GOING_ OUT'])
df6.head(20)
df6.shape

(89, 120)

###### Task 2. fill in missing data with new values

In [5]:
# MEH means no differnce so it could be used to fill in empty value for response.
#replace the NaN rating with MEH on column"100_Grand_bar and create a new column
df_temp = df6.iloc[:, 6].replace(np.nan, "MEH")

   # print(df.iloc[:,i])
    #df6.iloc[:,i] = o


###### Task 3. Merge 2 dataframe together 

In [6]:
#join the dataframe together
df_new =df6.iloc[:,0:6]
df_combined = df_new.join(df_temp)
df_combined

,Internal_ ID,GOING_ OUT,GENDER,AGE,COUNTRY,STATE_PROVINCE_ COUNTY,100_Grand_Bar
1,90272821,No,Male,44,USA,NM,MEH
3,90272840,No,Male,40,us,or,MEH
4,90272841,No,Male,23,usa,exton pa,JOY
6,90272853,No,Male,53,usa,Colorado,MEH
7,90272854,No,Male,33,canada,ontario,JOY
...,...,...,...,...,...,...,...
95,90273049,No,Male,50,United States,Maryland,JOY
96,90273050,No,Male,64,usa,alaska,MEH
97,90273051,No,Male,42,Usa,Ny,MEH
98,90273053,No,Female,35,USA,"Colorado, Jefferson county",MEH


###### Task 4 mapping

In [76]:
# create a mapping list to explain the response code:
response_explained = {
  'MEH': 'do not care',
  'JOY': 'like it',
  'DESPAIR': 'Do not like',
  }
df_combined['feedback comment'] = df_combined['100_Grand_Bar'].map(response_explained)
df_combined

,Internal_ ID,GOING_ OUT,GENDER,AGE,COUNTRY,STATE_PROVINCE_ COUNTY,100_Grand_Bar,feedback comment
1,90272821,No,Male,44,USA,NM,MEH,do not care
3,90272840,No,Male,40,us,or,MEH,do not care
4,90272841,No,Male,23,usa,exton pa,JOY,like it
6,90272853,No,Male,53,usa,Colorado,MEH,do not care
7,90272854,No,Male,33,canada,ontario,JOY,like it
...,...,...,...,...,...,...,...,...
95,90273049,No,Male,50,United States,Maryland,JOY,like it
96,90273050,No,Male,64,usa,alaska,MEH,do not care
97,90273051,No,Male,42,Usa,Ny,MEH,do not care
98,90273053,No,Female,35,USA,"Colorado, Jefferson county",MEH,do not care


##### End of exercise 7

##### Chapter 8 exercise:  hierarchical index and reshape data

In [ ]:
#use the df_combined data frame above

###### Task 5: Create hierarchical index

In [84]:
df_short =df_combined.iloc[:5, :5]
#df_short
new = df_short.stack()
new

1  Internal_ ID    90272821
   GOING_ OUT            No
   GENDER              Male
   AGE                   44
   COUNTRY             USA 
3  Internal_ ID    90272840
   GOING_ OUT            No
   GENDER              Male
   AGE                   40
   COUNTRY               us
4  Internal_ ID    90272841
   GOING_ OUT            No
   GENDER              Male
   AGE                   23
   COUNTRY              usa
6  Internal_ ID    90272853
   GOING_ OUT            No
   GENDER              Male
   AGE                   53
   COUNTRY              usa
7  Internal_ ID    90272854
   GOING_ OUT            No
   GENDER              Male
   AGE                   33
   COUNTRY           canada
dtype: object

###### Task 6. pivot the dataframe

In [102]:
pivoted = df_short.pivot('Internal_ ID', 'GENDER')
pivoted

#df = pd.DataFrame({'A': ['John', 'Boby', 'Mina'],
#                   'B': ['Masters', 'Graduate', 'Graduate'],
#                   'C': [27, 23, 21]})
#
#df.pivot('A', 'B', 'C')

C:\Users\Daisy\AppData\Local\Temp\ipykernel_1976\841386755.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  pivoted = df_short.pivot('Internal_ ID', 'GENDER')


,GOING_ OUT,AGE,COUNTRY
GENDER,Male,Male,Male
Internal_ ID,,,
90272821,No,44,USA
90272840,No,40,us
90272841,No,23,usa
90272853,No,53,usa
90272854,No,33,canada


##### End of chapter 8 exercise

##### Chapter 10 exercise: Grouping

In [7]:
# Create a new data frame with 2 candy feedback

df_temp2 = df6.iloc[:, 7].replace(np.nan, "MEH")
df_temp2

1     DESPAIR
3     DESPAIR
4     DESPAIR
6         MEH
7     DESPAIR
       ...   
95        MEH
96        MEH
97        MEH
98    DESPAIR
99    DESPAIR
Name: Mary_Janes, Length: 89, dtype: object

In [8]:
df_combined2 = df_combined.join(df_temp2)
df_combined2

,Internal_ ID,GOING_ OUT,GENDER,AGE,COUNTRY,STATE_PROVINCE_ COUNTY,100_Grand_Bar,Mary_Janes
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR
3,90272840,No,Male,40,us,or,MEH,DESPAIR
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR
6,90272853,No,Male,53,usa,Colorado,MEH,MEH
7,90272854,No,Male,33,canada,ontario,JOY,DESPAIR
...,...,...,...,...,...,...,...,...
95,90273049,No,Male,50,United States,Maryland,JOY,MEH
96,90273050,No,Male,64,usa,alaska,MEH,MEH
97,90273051,No,Male,42,Usa,Ny,MEH,MEH
98,90273053,No,Female,35,USA,"Colorado, Jefferson county",MEH,DESPAIR


###### Task 7: Grouping with Dicts/Series

In [17]:
#first group by gender and age
by_gender = df_combined2.groupby(['GENDER', 'AGE'])
by_gender.first()

Internal_ ID GOING_ OUT         COUNTRY  \
GENDER             AGE                                            
Female             32       90273017         No             USA   
                   33       90272904         No             Usa   
                   35       90273053         No             USA   
                   36       90272884         No   United States   
                   37       90272868         No          Canada   
                   38       90273023         No   United States   
                   40       90272995         No          Canada   
                   42       90272983         No             Usa   
                   44       90272962         No          Canada   
                   45       90272933         No             usa   
                   46       90272976         No             USA   
                   47       90272936         No             Usa   
                   48       90272924         No    United staes   
                   49       90272891         No   United States   
                   50       90272948         No   United States   
                   53       90272859         No              Us   
                   54       90272883         No   United States   
                   62       90272984         No             USA   
I'd rather not say 59       90272880         No             USA   
Male               16       90272906         No  United States    
                   23       90272841         No             usa   
                   25       90272894         No  United Kingdom   
                   27       90272935         No             USA   
                   28       90273045         No   United States   
                   29       90273000         No             USA   
                   31       90273032         No   United States   
                   33       90272854         No          canada   
                   34       90272895         No   united states   
                   35       90272897         No             USA   
                   36       90273011         No   United States   
                   37       90272887        Yes   United States   
                   38       90272898        Yes   United States   
                   39       90273008         No         England   
                   40       90272840         No              us   
                   41       90272934         No             USA   
                   42       90273047         No             USA   
                   43       90272867        Yes          Murica   
                   44       90272821         No            USA    
                   45       90272885         No              uk   
                   46       90272963         No             UAE   
                   47       90272905         No             USA   
                   48       90272881         No              US   
                   49       90272944         No   United States   
                   50       90272903        Yes             USA   
                   52       90272913         No   United States   
                   53       90272853         No             usa   
                   54       90272966         No             USA   
                   56       90272865         No          Canada   
                   58       90272899         No             USA   
                   59       90272947         No              US   
                   61       90272961         No             usa   
                   63       90272919         No   United States   
                   64       90272866         No              US   
                   69       90273048         No             USA   

                             STATE_PROVINCE_ COUNTY 100_Grand_Bar Mary_Janes  
GENDER             AGE                                                        
Female             32                          Iowa           JOY    DESPAIR  
                   33             

In [19]:
#then by dictionary
mapping = {'100_Grand_Bar': 'Bar', 'Mary_Janes': 'Mary'}
by_column = df_combined2.groupby(mapping, axis=1)
by_column.count()

,Bar,Mary
1,1,1
3,1,1
4,1,1
6,1,1
7,1,1
...,...,...
95,1,1
96,1,1
97,1,1
98,1,1


###### Task 8: group with function

In [27]:
#sort the dataframe by top 5 eldest, using a self-defined function: edlest

def eldest(df, n=5, column='AGE'):
    return df.sort_values(by=column)[-n:]
eldest(df_combined, n=5)


,Internal_ ID,GOING_ OUT,GENDER,AGE,COUNTRY,STATE_PROVINCE_ COUNTY,100_Grand_Bar
39,90272919,No,Male,63,United States,Wisconsin,JOY
15,90272878,No,Male,64,USA,Texas,JOY
12,90272866,No,Male,64,US,NY,MEH
96,90273050,No,Male,64,usa,alaska,MEH
94,90273048,No,Male,69,USA,Wisconsin,MEH


###### Task 9: Cross-Tabulations

In [30]:
#pd.crosstab(data.Gender, data.Handedness, margins=True)
pd.crosstab(df_combined.GENDER, df_combined.AGE, margins=True )

AGE,16,23,25,27,28,29,31,32,33,34,...,54,56,58,59,61,62,63,64,69,All
GENDER,,,,,,,,,,,,,,,,,,,,,
Female,0,0,0,0,0,0,0,1,1,0,...,1,0,0,0,0,2,0,0,0,24
I'd rather not say,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
Male,1,1,1,1,1,1,1,0,3,5,...,1,2,1,1,1,0,1,3,1,64
All,1,1,1,1,1,1,1,1,4,5,...,2,2,1,2,1,2,1,3,1,89


###### End of chapter 10 Exercise

##### Chapter 11 Exercise: Data range and time series

In [35]:
# this one I will use Museum of Art data, I cut it to 500 rows to save memory
df_museum = pd.read_excel('MetObjects-lean.xlsx')
df_museum.head()
df_museum.shape

(550, 32)

###### Task10. Convert between string and date time

In [33]:
#read the year, which is a float64 variable, convert it to datetime type
from datetime import timedelta, datetime
A_year = df_museum['AccessionYear']
df_DT = pd.to_datetime(df_museum['AccessionYear'], format='%Y')
print(df_DT)

# covert to string
print(str(df_DT[0]))
print('\n the range of collection is:',df_DT.max()-df_DT.min())

0     1979-01-01
1     1980-01-01
2     1967-01-01
3     1967-01-01
4     1967-01-01
         ...    
545   1940-01-01
546   1940-01-01
547   1940-01-01
548   1940-01-01
549   1940-01-01
Name: AccessionYear, Length: 550, dtype: datetime64[ns]
1979-01-01 00:00:00

 the range of collection is: 47847 days 00:00:00


###### Task11. Generate date range

In [41]:
df_DT.truncate(after='1/1/1960')

0   1979-01-01
1   1980-01-01
Name: AccessionYear, dtype: datetime64[ns]

###### Task 12. Frequencies and Date Offsets

In [45]:
#offset by 10 hours
from pandas.tseries.offsets import Hour, Minute
new_date = df_DT + Hour(10)
print(new_date[0:5])

0   1979-01-01 10:00:00
1   1980-01-01 10:00:00
2   1967-01-01 10:00:00
3   1967-01-01 10:00:00
4   1967-01-01 10:00:00
Name: AccessionYear, dtype: datetime64[ns]


##### End of chapter 11 exercise